# Question
_Bootstrap with Least squares, Ridge and Lasso._ Let $\beta = (\beta_1, \beta_2, \cdots , \beta_p)$ and let $\pmb x, y$ be random variables such that the entries of $\pmb x$ are i.i.d. standard normal random variables (i.e., with mean zero and variance $1$) and $y = \beta^T \pmb x + \epsilon$ where $\epsilon \sim \mathcal N(0, 1)$.
\begin{enumerate}
\item Simulate a dataset $(\pmb x_1, y_1), \cdots, (\pmb x_n, y_n)$ as $n$ i.i.d. copies of the random variables $\pmb x, y$ defined above, with $n = 800, p = 200$ and $\beta_j = j^{−1}$:
\item The goal of this problem is to construct confidence intervals for $\beta_1$ using Bootstrap method.
\begin{enumerate}[label=(\alph*)]
\item Construct confidence intervals for $\beta_1$ by bootstrapping the data and applying Least Squares to the bootstrapped dataset.
\item Construct confidence intervals for $\beta_1$ by bootstrapping the data and applying Ridge to the bootstrapped data set.
\item Construct confidence intervals for $\beta_1$ by bootstrapping the data and applying Lasso to the bootstrapped data set.
\end{enumerate}\end{enumerate}

First we initialize the above $n,p,\beta$ as given in the above problem.

In [89]:
n = 800
p = 200
b = 1/(1:p)

Now let's generate data:

In [90]:
x = c()
for(i in 1:n){
    x = rbind(x,rnorm(p))
}
y = x %*% b + rnorm(n)

We now bootstrap, to get confidence intervals for $\beta_1$. We sample $B$ datapoints from the above population.

**Least squares:** The estimated $\hat{\pmb\beta} = \left(\hat\beta_1,\cdots,\hat\beta_p\right)$ is given by $M=\left(X^\top X\right)^{-1}X^\top Y$. We note that $\hat\beta_1$ is just the first entry of $M$. But this is just $M_1=\left[\left(X^\top X\right)^{-1}X^\top Y\right]_1 = \left[\left(X^\top X\right)^{-1}X^\top \right]_{1:}\cdot Y = \left[\left(X^\top X\right)^{-1}\right]_{1:}  \cdot X^\top \cdot Y$. Here $A_{i:}$ denotes the $i^{\text{th}}$ row of $A$. 

In [91]:
library(glmnet)
B = 1
b1.hat_ls = c()
b1.hat_lasso = c()
b1.hat_ridge = c()
for(i in 1:B){
    index = sample.int(n, n, replace = TRUE)
    x.hat = x[index,]
    y.hat = y[index]
            
    #use least squares to estimate beta
    M = t(x.hat) %*% x.hat
    M = solve(M)[1,] %*% t(x.hat)
    M = M %*% y.hat
    b1.hat_ls = append(b1.hat_ls, M)
    lasso = glmnet(x.hat,y.hat,family = 'gaussian')
}
print(lasso)
b1.hat_ls = sort(b1.hat_ls)
print(b1.hat_ls)


Call:  glmnet(x = x.hat, y = y.hat, family = "gaussian") 

    Df  %Dev  Lambda
1    0  0.00 0.94350
2    1  5.78 0.85970
3    1 10.58 0.78330
4    1 14.56 0.71370
5    1 17.87 0.65030
6    1 20.62 0.59260
7    2 23.54 0.53990
8    2 27.45 0.49200
9    2 30.69 0.44820
10   2 33.38 0.40840
11   2 35.62 0.37210
12   3 37.70 0.33910
13   3 39.96 0.30900
14   3 41.84 0.28150
15   4 43.86 0.25650
16   4 45.54 0.23370
17   6 47.34 0.21300
18   6 49.07 0.19400
19   6 50.51 0.17680
20   8 51.88 0.16110
21  10 53.18 0.14680
22  11 54.41 0.13370
23  14 55.53 0.12190
24  19 56.82 0.11100
25  20 58.17 0.10120
26  22 59.45 0.09218
27  32 60.81 0.08399
28  36 62.15 0.07653
29  40 63.50 0.06973
30  46 64.75 0.06354
31  59 66.02 0.05789
32  63 67.27 0.05275
33  71 68.44 0.04806
34  75 69.48 0.04379
35  80 70.41 0.03990
36  85 71.30 0.03636
37  94 72.11 0.03313
38 100 72.93 0.03019
39 110 73.70 0.02750
40 116 74.40 0.02506
41 122 75.02 0.02283
42 128 75.59 0.02081
43 131 76.09 0.01896
44 132 76.52 0.0

Now that we have the (sorted) bootstrapped data, we want to find the confidence intervals. Recall that $(1-\alpha)^{\text{th}}\cdot 100\%$ interval for $\beta_1$ is $\left[\hat\theta_{[L]}^*,\hat\theta_{[U]}^*\right]$ where $L=\frac{\alpha B}{2}, U=\left(1-\frac{\alpha}{2}\right)B$ and $\hat\theta_i^*$ are the sorted bootstrapped data.

In [53]:
a = 0.05
L = floor(a * B / 2)
U = floor((1 - a / 2) * B)
print(b1.hat_ls[L])
print(b1.hat_ls[U])

numeric(0)
numeric(0)
